In [1]:
import numpy as numpy
import torch 
from model_dynamic_streaming_CP import LDS_dynammic_streaming
import utils_streaming
import tqdm
import yaml

path = "base.yaml"
with open(path, "r") as f:
    config = yaml.safe_load(f)

data_file = '../processed_data/beijing_15k.npy'
# data_file = '../processed_data/mvlens_10k.npy'

hyper_dict = utils_streaming.make_hyper_dict(config)
data_dict = utils_streaming.make_data_dict(hyper_dict,data_file)

model =LDS_dynammic_streaming(hyper_dict, data_dict)

use device: cpu


In [2]:
model.time_uni = model.time_uni*10
for T in range(model.N_time):
    model.track_envloved_objects(T)
    model.filter_predict(T)
    model.msg_approx(T)
    model.filter_update(T)

# model.smooth()
# model.get_post_U()

In [3]:
test_result = model.model_test(model.ind_te,model.y_te,model.test_time_ind)
test_result

tensor(0.9542, dtype=torch.float64)

In [8]:
test_result

tensor(0.9542, dtype=torch.float64)

In [3]:
traj = model.traj_class[1][0]
traj.m_pred_list


[tensor([[ 0.1559],
         [-0.1607]], dtype=torch.float64),
 tensor([[-0.0055],
         [-0.0589]], dtype=torch.float64),
 tensor([[ 0.0034],
         [-0.0767]], dtype=torch.float64),
 tensor([[ 0.0188],
         [-0.0578]], dtype=torch.float64),
 tensor([[ 0.0212],
         [-0.0461]], dtype=torch.float64),
 tensor([[ 0.0074],
         [-0.0440]], dtype=torch.float64),
 tensor([[ 0.0064],
         [-0.0421]], dtype=torch.float64),
 tensor([[ 0.0052],
         [-0.0385]], dtype=torch.float64),
 tensor([[ 0.0046],
         [-0.0370]], dtype=torch.float64),
 tensor([[ 0.0041],
         [-0.0360]], dtype=torch.float64),
 tensor([[ 0.0084],
         [-0.0376]], dtype=torch.float64),
 tensor([[ 0.0079],
         [-0.0350]], dtype=torch.float64),
 tensor([[ 0.0072],
         [-0.0332]], dtype=torch.float64),
 tensor([[ 0.0068],
         [-0.0314]], dtype=torch.float64),
 tensor([[ 0.0064],
         [-0.0298]], dtype=torch.float64),
 tensor([[ 0.0061],
         [-0.0282]], dtype=torch.fl

In [5]:
traj.P_inf

tensor([[1., 0.],
        [0., 1.]], dtype=torch.float64)

In [15]:
traj.m_update_list

[tensor([[0.2843],
         [0.0931]], dtype=torch.float64),
 tensor([[0.1627],
         [0.0512]], dtype=torch.float64),
 tensor([[0.1377],
         [0.0268]], dtype=torch.float64),
 tensor([[0.1243],
         [0.0205]], dtype=torch.float64),
 tensor([[0.1017],
         [0.0016]], dtype=torch.float64),
 tensor([[0.0951],
         [0.0039]], dtype=torch.float64),
 tensor([[0.0828],
         [0.0033]], dtype=torch.float64),
 tensor([[ 0.0592],
         [-0.0007]], dtype=torch.float64),
 tensor([[0.0539],
         [0.0004]], dtype=torch.float64),
 tensor([[ 0.0513],
         [-0.0020]], dtype=torch.float64),
 tensor([[ 0.0471],
         [-0.0021]], dtype=torch.float64),
 tensor([[ 0.0444],
         [-0.0019]], dtype=torch.float64),
 tensor([[ 0.0422],
         [-0.0018]], dtype=torch.float64),
 tensor([[ 0.0391],
         [-0.0017]], dtype=torch.float64),
 tensor([[ 0.0364],
         [-0.0016]], dtype=torch.float64),
 tensor([[ 0.0338],
         [-0.0015]], dtype=torch.float64),
 tensor(

In [16]:
# mode = 0
# model.LDS_list[mode].reset_list()

# for k in range(model.N_time):

#     # update model.msg_U_M, model.msg_U_V : compute Ez, Ez2, get \beta, S, emseble to long vec msg_U
#     model.msg_update_U(mode, k)

#     # Kalman Filter: predict and update
#     model.LDS_list[mode].filter_predict(ind=k)
#     model.LDS_list[mode].filter_update(y=model.msg_U_M, R=model.msg_U_V)

# model.LDS_list[mode].smooth()
# model.post_update_U(mode)

In [7]:
for i in tqdm.tqdm(range(30)):

    for mode in range(model.nmods):

        model.LDS_list[mode].reset_list()
        
        for k in range(model.N_time):

            # update model.msg_U_M, model.msg_U_V : compute Ez, Ez2, get \beta, S, emseble to long vec msg_U
            model.msg_update_U(mode, k)

            # Kalman Filter: predict and update
            model.LDS_list[mode].filter_predict(ind=k)
            model.LDS_list[mode].filter_update(y=model.msg_U_M, R=model.msg_U_V)

        model.LDS_list[mode].smooth()

        model.post_update_U(mode)

    test_result = model.model_test(model.ind_te, model.y_te, model.test_time_ind).cpu().numpy()

    print("epoch:", i, 'test-rmse:',test_result)

  0%|          | 0/30 [00:00<?, ?it/s]

In [8]:
model.post_U_m[2]

tensor([[[[ 5.1251e-03,  5.0288e-03,  4.9276e-03,  ..., -4.9635e-04,
           -3.7468e-04, -2.5580e-04]],

         [[ 8.8006e-03,  8.6875e-03,  8.5687e-03,  ...,  2.4943e-04,
            1.3882e-04,  3.0789e-05]]],


        [[[ 6.2479e-03,  6.1392e-03,  6.0250e-03,  ...,  1.5002e-04,
            1.5778e-04,  1.6534e-04]],

         [[ 6.8264e-03,  6.7113e-03,  6.5906e-03,  ..., -2.8678e-04,
           -3.3470e-04, -3.8145e-04]]],


        [[[ 6.5536e-03,  6.4386e-03,  6.3181e-03,  ...,  2.8377e-04,
            2.9791e-04,  3.1167e-04]],

         [[ 7.1047e-03,  6.9885e-03,  6.8667e-03,  ..., -2.2384e-04,
           -2.5410e-04, -2.8363e-04]]],


        ...,


        [[[ 5.4756e-03,  5.3760e-03,  5.2714e-03,  ...,  7.3272e-05,
            1.8894e-04,  3.0186e-04]],

         [[ 6.7351e-03,  6.6200e-03,  6.4993e-03,  ..., -3.7380e-04,
           -4.6509e-04, -5.5417e-04]]],


        [[[ 5.5856e-03,  5.4909e-03,  5.3913e-03,  ...,  6.2971e-05,
            4.9768e-05,  3.6863e-05]

In [11]:
model.msg_U_M


tensor([[ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 1.1809e-05],
        [ 4.0384e-06],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [-1.0967e-06],
        [ 8.2535e-07],
        [ 0.0000e+00],
        [ 0.0000e+00]], device='cuda:0')

In [11]:
test_result.cpu().numpy()

array(0.95424135)

In [7]:
import numpy as np
a = np.zeros((2,2))
len(a.shape)

2